# Etiquetado de imagenes para el entrenamiento del Modelo basado en YoloV8

## Definiendo funciones utiles necesarias

In [20]:
# Definiendo la clase utils para usar funciones genericas
import numpy as np
import cv2 as cv

class Utils:

    @staticmethod
    def filterKernel():
        return np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])


## Creación de las opciones básicas que usa el programa


In [21]:
import tkinter as tk
from tkinter import *
from tkinter import filedialog, ttk

class OptionsHough:
    def __init__(self):
        self.minRadius = "25"
        self.maxRadius = "45"
        self.param1 = "80"
        self.param2 = "35"
        self.mediumCellSize = "180"
        self.scale = "0.59"
        self.cameraHeight = "10"
        self.diluent = "0"        

    def return_options(self):
        return(int(self.minRadius),
                                       int(self.maxRadius),
                                       int(self.param1),
                                       int(self.param2),
                                       int(self.mediumCellSize),
                                       float(self.scale),
                                       int(self.cameraHeight),
                                       int(self.diluent)
                                       )

## Definición de la clase FrameVideo

In [22]:
class FrameVideo():

    def __init__(self, img, number_frame):
        self.circles_opencv = []
        self.number_frame = number_frame
        self.frame = img
        
    @staticmethod
    def analyzeFrame(frame_and_options):
        frame, options = frame_and_options
        filterKernel = Utils.filterKernel()
        (minRadius, maxRadius, _, _, _, _, _, _) = options


        height, width, depth = frame.frame.shape
        w = width + 50
        h = height + 50

        image = cv.cvtColor(frame.frame, cv.COLOR_BGR2GRAY)
        cv.normalize(image, image, 0, 255, cv.NORM_MINMAX)
        image = cv.filter2D(image, -1, filterKernel)
        img_border = cv.copyMakeBorder(image, 50, 50, 50, 50, cv.BORDER_REFLECT)
        circles = cv.HoughCircles(img_border, cv.HOUGH_GRADIENT, 1, 25, param1=80, param2=35, minRadius=minRadius,
                                  maxRadius=maxRadius)
        circles = np.uint16(np.around(circles))
        # I choose the circles that are inside the image
        circles2 = []
        for (x, y, r) in circles[0, :]:
            if ((x >= 50) and x <= w and (y >= 50) and y <= h):
                mask = np.zeros((height, width), np.uint8)
                cv.circle(mask, (x, y), r, 255, -1)
                points = np.transpose(np.where(mask == 255))
                sum = 0
                for (a, b) in points:
                    sum = sum + img_border[a][b]
                # suma = suma + img_border[a][b][0] + img_border[a][b][1] + img_border[a][b][2] # Aqui mejor poner algo como que > 50% pixeles tienen alta intensidad
                if (sum <= 128 * 0.6 * len(points)):
                    circles2.append([x - 50, y - 50, r])

        frame.circles_opencv = circles2
        frame.frame=image
        return frame

## Definición clase video Analyzer

In [25]:
from multiprocessing import Pool
import os

class VideoAnalyzer:

    def __init__(self, file_path):
        self.file_path = file_path
        self.output_dir = os.path.join(os.path.dirname(file_path), "Output_" + os.path.splitext(os.path.basename(file_path))[0])
        os.makedirs(self.output_dir, exist_ok=True)
        self.frames = []

    def videoSpermDetection(self, options):
        self.frames = [] # Reiniciamos por si acaso
        video = self.file_path

        # Creamos las carpetas de salida
        output_dir_images_rect = os.path.join(self.output_dir, "Images_Rectangles")
        output_dir_labels = os.path.join(self.output_dir, "Labels")
        output_dir_images_original = os.path.join(self.output_dir, "Images_Original")
        os.makedirs(output_dir_images_rect, exist_ok=True)
        os.makedirs(output_dir_labels, exist_ok=True)
        os.makedirs(output_dir_images_original, exist_ok=True)

        vidcap = cv.VideoCapture(video)
        success, image = vidcap.read()
        frame_number = 0

        while success:
            f = FrameVideo(image, frame_number)
            self.frames.append(f)
            success, image = vidcap.read()
            frame_number += 1

        vidcap.release()

        for frame in self.frames:
            frame_and_options = (frame, options)
            analyzed_frame = FrameVideo.analyzeFrame(frame_and_options)
            self.frames[frame.number_frame] = analyzed_frame

        for frame in self.frames:
            # Convertir el marco a color
            frame.frame = cv.cvtColor(frame.frame, cv.COLOR_GRAY2BGR)

            # Guardar la imagen original
            image_filename_original = os.path.join(output_dir_images_original, f"frame_{frame.number_frame}.jpg")
            cv.imwrite(image_filename_original, frame.frame)

            # Dibujar cuadrados alrededor de los círculos y guardar la imagen con los cuadrados dibujados
            for i, circle in enumerate(frame.circles_opencv, start=1):
                x, y, r = circle
                # Aumentar el radio en 2 píxeles
                r += 2
                x_left = x - r
                y_top = y - r
                x_right = x + r
                y_bottom = y + r
                cv.rectangle(frame.frame, (x_left, y_top), (x_right, y_bottom), (0, 255, 0), 2)
                cv.putText(frame.frame, str(i), (x_left + 5, y_bottom - 5), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

            image_filename_rect = os.path.join(output_dir_images_rect, f"frame_{frame.number_frame}.jpg")
            cv.imwrite(image_filename_rect, frame.frame)

            # Guardar las etiquetas YOLO en un archivo
            label_filename = os.path.join(output_dir_labels, f"frame_{frame.number_frame}.txt")
            with open(label_filename, "w") as f:
                for i, circle in enumerate(frame.circles_opencv, start=1):
                    x, y, r = circle
                    # Aumentar el radio en 2 píxeles
                    r += 2
                    # Normalizar las coordenadas X y Y y calcular el ancho y alto del cuadro
                    img_width = frame.frame.shape[1]
                    img_height = frame.frame.shape[0]
                    x_center = x / img_width
                    y_center = y / img_height
                    box_width = (2 * r) / img_width
                    box_height = (2 * r) / img_height
                    f.write(f"{i} {x_center:.3f} {y_center:.3f} {box_width:.3f} {box_height:.3f} {i}\n")

In [26]:
video1 = VideoAnalyzer("D:\OneDrive - Universidad de La Rioja\RepositoriosPersonales\TrabajosMaestriaCienciadeDatos\Proyecto_TFM\Supplementary_Files\Supplementary_Files\Suppl_1.avi")
video2 = VideoAnalyzer("D:\OneDrive - Universidad de La Rioja\RepositoriosPersonales\TrabajosMaestriaCienciadeDatos\Proyecto_TFM\Supplementary_Files\Supplementary_Files\Suppl_2.avi")
video3 = VideoAnalyzer("D:\OneDrive - Universidad de La Rioja\RepositoriosPersonales\TrabajosMaestriaCienciadeDatos\Proyecto_TFM\Supplementary_Files\Supplementary_Files\Suppl_3.avi")
video4 = VideoAnalyzer("D:\OneDrive - Universidad de La Rioja\RepositoriosPersonales\TrabajosMaestriaCienciadeDatos\Proyecto_TFM\Supplementary_Files\Supplementary_Files\Suppl_4.avi")
video5 = VideoAnalyzer("D:\OneDrive - Universidad de La Rioja\RepositoriosPersonales\TrabajosMaestriaCienciadeDatos\Proyecto_TFM\Supplementary_Files\Supplementary_Files\Suppl_5.avi")
video6 = VideoAnalyzer("D:\OneDrive - Universidad de La Rioja\RepositoriosPersonales\TrabajosMaestriaCienciadeDatos\Proyecto_TFM\Supplementary_Files\Supplementary_Files\Suppl_6.avi")
options = OptionsHough()
video1.videoSpermDetection(options.return_options())
video2.videoSpermDetection(options.return_options())
video3.videoSpermDetection(options.return_options())
video4.videoSpermDetection(options.return_options())
video5.videoSpermDetection(options.return_options())
video6.videoSpermDetection(options.return_options())